In [170]:
from pathlib import Path
import pandas as pd
import os

from delphi_nhsn.constants import SIGNALS_MAP, PRELIM_SIGNALS_MAP
from delphi_epidata import Epidata
from epiweeks import Week
API_KEY = os.environ.get('DELPHI_API_KEY')

CSV_PATH = Path(os.path.abspath('')).parent.joinpath('receiving')
SIGNALS = list(SIGNALS_MAP.keys()) + list(PRELIM_SIGNALS_MAP.keys())
Epidata.auth = ("epidata", API_KEY)


In [174]:
def get_nhsn_df(geo, pathogen, prelim=False):
    lst = []
    geo_filter = f'*{geo}'
    pathogen_filter = f'confirmed_admissions_{pathogen}.csv'
    prelim_filter = f'prelim_' if prelim else ""
    filter = f"{geo_filter}_{prelim_filter}{pathogen_filter}"
    for files in Path(CSV_PATH).glob(filter):
        filename_parts = files.name.split('_')
        signal = "_".join(filename_parts[3:]).replace('.csv', '')
        time_value = filename_parts[1]
        df = pd.read_csv(files)
        df['signal'] = signal
        df['time_value'] = time_value
        df["geo_id"] = df["geo_id"].astype(str)
        if geo == "nation":
            df["geo_id"] = "US"
        elif geo == "state":
            df["geo_id"] = df["geo_id"].str.upper()
        df.time_value = df.time_value.astype(str)
        lst.append(df)
    df = pd.concat(lst)
    df = df.drop(columns=['se','sample_size'], axis=1)
    return df

In [175]:
def get_hhs_df(geo, pathogen):
    response = Epidata.covidcast("hhs", f"confirmed_admissions_{pathogen}_1d",
                      geo_type=geo, time_values=Epidata.range("20200808", "20240426"),
                              geo_value="*", as_of=None, time_type="day")
    df = pd.DataFrame.from_dict(response["epidata"])
    df.rename(columns={"geo_value": "geo_id", "value": "val", "stderr": "se"}, inplace=True)
    df = df[["geo_id", "time_value", "val", "signal"]]
    df["time_value"] = pd.to_datetime(df["time_value"], format="%Y%m%d")
    # aggregating over 7 days matching epiweek for nhsn
    df["week_time"] = df["time_value"].apply(lambda x: Week.fromdate(x))
    df = df.groupby(["geo_id", "signal", "week_time"]).val.sum().reset_index()
    df["geo_id"] = df["geo_id"].astype(str)
    if geo != "hhs":
        df["geo_id"] = df["geo_id"].str.upper()
    df.rename({"week_time": "time_value"}, axis=1, inplace=True)
    df.time_value = df.time_value.astype(str)
    return df


In [ ]:
def generate_spearman(nhsn_df, hhs_df, slice_col):
    
    joined = pd.merge(nhsn_df, hhs_df, on=["geo_id", "time_value"], how="left", suffixes=["_nhsn", "_hhs"])
    
    spearmanr_df = joined.groupby(slice_col)[["val_nhsn", "val_hhs"]].corr("spearman").iloc[0::2,-1].reset_index()
    spearmanr_df.rename({"val_hhs": "correlation"}, axis=1, inplace=True)
    spearmanr_df = spearmanr_df[[slice_col, "correlation"]]
    spearmanr_df[slice_col] = spearmanr_df[slice_col].str.upper()
    return spearmanr_df
    

In [177]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

from plotly.offline import init_notebook_mode

init_notebook_mode(connected=True)

def plot_spearman_choropleth(nhsn_df, hhs_df, pathogen, prelim_flag, geo_val, download=False):
    spearmanr = generate_spearman(nhsn_df, hhs_df, "geo_id")
    title = f'Spearman Correlations between {geo_val} HHS {pathogen} admission and NHSN {pathogen} admission'
    filename = f'spearman_choropleth_{pathogen}_{geo_val}'
    
    if prelim_flag:
        title = f"{title} (prelim data)"
        filename = f"{filename}_prelim"
    
    filename = filename + ".pdf"
        
    fig = go.Figure(data=go.Choropleth(
        locations=spearmanr['geo_id'], # Spatial coordinates
        z = spearmanr['correlation'].astype(float), # Data to be color-coded
        locationmode = 'USA-states', # set of locations match entries in `locations`
        colorscale = 'Viridis',
        colorbar_title = "spearman rank",
    ))
    
    fig.update_layout(
        title_text = title,
        geo_scope='usa', # limite map scope to USA
    )
    
    if download:
        pio.write_image(fig, filename, format='pdf')
    
    else:
        fig.show()


In [178]:
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode
import plotly.io as pio

init_notebook_mode(connected=True)

def plot_timeplot(df, pathogen, prelim_flag, geo_val, y_col,download=False):
    if y_col == "correlation":
        title = f'Spearman Correlations between {pathogen} {geo_val} HHS and NHSN over time'
        filename = f'spearman_timeplot_{pathogen}_{geo_val}'
    else:
        title = f'{geo_val} NHSN value over time'
        filename = f'{geo_val}_{pathogen}_nhsn_timeplot'
    
    if prelim_flag:
        filename = f"{filename}_prelim"
        title = f"{title} (prelim data)"
    
    filename = filename + ".pdf"
    
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df['time_value'], y=df[y_col], mode='lines', name='Time Series'))
    fig.update_layout(title=title)

    
    if download:
        pio.write_image(fig, filename, format='pdf')
    else:
        fig.show()

In [184]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
from plotly.offline import init_notebook_mode

init_notebook_mode(connected=True)

geo_type = "state"
pathogen = "covid"
nhsn_df = get_nhsn_df(geo_type, pathogen)

def plot_choropleth_overtime(df, pathogen, prelim_flag, geo_val, z_col):
    color_bar_title = ""
    layout_title = "" 
    if z_col == "val" :
        color_bar_title = f'{geo_val} Total Confirmed {pathogen} Cases'
        layout_title = f'{geo_val} NHSN total confirmed {pathogen} cases' 

    elif z_col == "diff_percentage":
        color_bar_title = f"{geo_val} difference in percentage"
        layout_title = f'Proportion between NHSN and HHS for {geo_val} confirmed {pathogen} over time'
    
    if prelim_flag:
        layout_title = f'{layout_title} (prelim data)'
        
    df["time_value"] = pd.to_datetime(df.time_value.map(lambda x: str(x)+'-0'), format="%Y%W-%w")
    
    data_slider = []
    
    for time_value in df['time_value'].unique():
        df_segmented =  df[(df['time_value']== time_value)]
    
        data_each_yr = dict(
                            type='choropleth',
                            locations = df_segmented['geo_id'],
                            z=df_segmented[z_col].astype(float),
                            locationmode='USA-states',
                            colorscale = 'Viridis',
                            colorbar= {'title': color_bar_title},)
    
        data_slider.append(data_each_yr)
    
    steps = []
    for idx, time_value in enumerate(sorted(df['time_value'].unique())):
        step = {
                    "method":'restyle',
                    "args":[
                        {'visible': [False] * len(data_slider)}
                    ],
                    "label": f"Epiweek {Week.fromdate(pd.to_datetime(time_value))}"
                }
        step['args'][0]["visible"][idx] = True
        steps.append(step)
    
    sliders = [dict(active=0, pad={"t": 1}, steps=steps)]
    
    layout = dict(title =layout_title, geo=dict(scope='usa',
                           projection={'type': 'albers usa'}),
                  sliders=sliders)
    
    fig = go.Figure(data=data_slider, layout=layout)
    
    fig.show()


In [180]:
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode
import plotly.io as pio

init_notebook_mode(connected=True)

def plot_compare_timeplot(nhsn_df, hhs_df, pathogen, prelim_flag="False", geo_val="", download=False, val_cutoff=100, percent_cutoff=20):
    """
    
    Parameters
    ----------
    nhsn_df
    hhs_df
    pathogen
    geo_val
    download
    val_cutoff: min value to consider for suspicous
    percent_cutoff: min percentage of difference between NHSN and HHS to consider for suspicous

    Returns
    -------

    """
    title = f'{geo_val} NHSN vs HHS value over time with (diff > {percent_cutoff}% and val > {val_cutoff}) markers'
    filename = f'nhsn_vs_hhs_timeplot_{geo_val}_{pathogen}'
    
    if prelim_flag:
        title = f"{title} (prelim data)"
        filename = f"{filename}_prelim"

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=nhsn_df['time_value'], y=nhsn_df["val"], mode='lines', name='NHSN Time Series'))
    fig.add_trace(go.Scatter(x=hhs_df['time_value'], y=hhs_df["val"], mode='lines', name='HHS Time Series'))
    
    joined = pd.merge(nhsn_df, hhs_df, on=["geo_id", "time_value"], how="left", suffixes=["_nhsn", "_hhs"])
    joined = joined[["geo_id", "time_value", "val_nhsn", "val_hhs"]]
    joined["diff_percentage"] = ((joined["val_nhsn"] - joined["val_hhs"]) / joined["val_hhs"]) * 100
    percent_filter = (abs(joined["diff_percentage"]) > percent_cutoff)
    num_filter = (joined['val_nhsn'] > val_cutoff)
    sus_data = joined[(percent_filter & num_filter)]
    sus_data.sort_values(by="time_value", inplace=True)
    sus_data.to_csv(f"{filename}.csv", index=False)
    
    fig.add_trace(go.Scatter(x=sus_data['time_value'], y=sus_data["val_nhsn"] , mode='markers', name=f'NHSN points', marker={"color":"green"}))
    fig.add_trace(go.Scatter(x=sus_data['time_value'], y=sus_data["val_hhs"] , mode='markers', name=f'HHS points', marker={"color":"green"}))

    fig.update_layout(title=title)
    
    if download:
        pio.write_image(fig, f"{filename}.pdf", format='pdf')
    else:
        fig.show()


# all analysis download

In [183]:
from itertools import product
geos = ["nation", "state", "hhs"]
hhs_pathogen_mapping = {"flu": "influenza", "covid": "covid"}
geo_pathogen_flag_combo = list(product(geos, hhs_pathogen_mapping.keys(), [True, False]))
percent_cutoff = 30
val_cutoff = 100

for geo, pathogen, prelim_flag in geo_pathogen_flag_combo:
    nhsn_df = get_nhsn_df(geo, pathogen, prelim_flag)
    hhs_df = get_hhs_df(geo, hhs_pathogen_mapping[pathogen])
    spearmanr = generate_spearman(nhsn_df, hhs_df, "time_value")
    joined = pd.merge(nhsn_df, hhs_df, on=["geo_id", "time_value"], how="left", suffixes=["_nhsn", "_hhs"])
    joined = joined[["geo_id", "time_value", "val_nhsn", "val_hhs"]]
    joined["diff_percentage"] = ((joined["val_nhsn"] - joined["val_hhs"]) / joined["val_hhs"]) * 100
    
    plot_timeplot(spearmanr, pathogen, prelim_flag, geo, y_col="correlation", download=True)
    plot_timeplot(nhsn_df, pathogen, prelim_flag, geo, y_col="val", download=True)
    plot_timeplot(joined, pathogen, prelim_flag, geo, y_col="diff_percentage", download=True)
    if geo == "state":
        plot_spearman_choropleth(nhsn_df, hhs_df, pathogen, prelim_flag, geo, download=True)
    
    if geo != "nation":
        geos = list(nhsn_df.geo_id.unique())
        for geo_val in geos:
            part_nhsn_df = nhsn_df[nhsn_df["geo_id"] == geo_val].sort_values(by="time_value")
            part_hhs_df = hhs_df[hhs_df["geo_id"] == geo_val].sort_values(by="time_value")

            plot_compare_timeplot(part_nhsn_df, part_hhs_df, pathogen, prelim_flag, geo_val=geo_val, val_cutoff=val_cutoff, percent_cutoff=percent_cutoff, download=True)
    else:
        plot_compare_timeplot(nhsn_df, hhs_df, pathogen, prelim_flag, geo_val="us", val_cutoff=val_cutoff, percent_cutoff=percent_cutoff, download=True)

/var/folders/s6/ytv8f9z11q32lhh5ksxrp_tm0000gn/T/ipykernel_43215/2215692234.py:41: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/s6/ytv8f9z11q32lhh5ksxrp_tm0000gn/T/ipykernel_43215/2215692234.py:41: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/s6/ytv8f9z11q32lhh5ksxrp_tm0000gn/T/ipykernel_43215/2215692234.py:41: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/s6/ytv8f9z11q32lhh5ksxrp_tm0000gn/T/ipy

# Spearman over time example

In [ ]:
from itertools import product

pathogen_mapping = {"flu": "influenza", "covid": "covid"}
pathogen = "covid"
prelim_flag = False
geo = "state"

nhsn_df = get_nhsn_df(geo, pathogen, prelim_flag)
hhs_df = get_hhs_df(geo, pathogen_mapping[pathogen])
spearmanr = generate_spearman(nhsn_df, hhs_df, "time_value")
plot_timeplot(spearmanr, pathogen, prelim_flag, geo, y_col="correlation", download=True)


# Spearman over geo

In [ ]:
pathogen_mapping = {"flu": "influenza", "covid": "covid"}
pathogen = "covid"
prelim_flag = False
geo = "state"

nhsn_df = get_nhsn_df(geo, pathogen, prelim_flag)
hhs_df = get_hhs_df(geo, pathogen_mapping[pathogen])
plot_spearman_choropleth(nhsn_df, hhs_df, pathogen, prelim_flag, geo, download=True)


# comparison over time across geo

In [185]:
pathogen_mapping = {"flu": "influenza", "covid": "covid"}
pathogen = "covid"
prelim_flag = False
geo = "state"

nhsn_df = get_nhsn_df(geo, pathogen, prelim_flag)
hhs_df = get_hhs_df(geo, pathogen_mapping[pathogen])
joined = pd.merge(nhsn_df, hhs_df, on=["geo_id", "time_value"], how="left", suffixes=["_nhsn", "_hhs"])
joined = joined[["geo_id", "time_value", "val_nhsn", "val_hhs"]]
joined["diff_percentage"] = ((joined["val_nhsn"] - joined["val_hhs"]) / joined["val_hhs"]) * 100
plot_choropleth_overtime(joined, pathogen, prelim_flag, geo, z_col="diff_percentage")

# compare val over time

In [131]:
pathogen_mapping = {"flu": "influenza", "covid": "covid"}
pathogen = "covid"
prelim_flag = False
geo = "state"
val_cutoff = 100
percent_cutoff = 30


nhsn_df = get_nhsn_df(geo, pathogen).sort_values(by="time_value")
hhs_df = get_hhs_df(geo, pathogen_mapping[pathogen]).sort_values(by="time_value")
geos = list(nhsn_df.geo_id.unique())
for geo_val in geos[:2]:
    state_nhsn_df = nhsn_df[nhsn_df["geo_id"] == geo_val].sort_values(by="time_value")
    state_hhs_df = hhs_df[hhs_df["geo_id"] == geo_val].sort_values(by="time_value")
    plot_compare_timeplot(nhsn_df, hhs_df, pathogen, prelim_flag, geo_val=geo_val, val_cutoff=val_cutoff, percent_cutoff=percent_cutoff, download=False)


/var/folders/s6/ytv8f9z11q32lhh5ksxrp_tm0000gn/T/ipykernel_43215/3830264213.py:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



/var/folders/s6/ytv8f9z11q32lhh5ksxrp_tm0000gn/T/ipykernel_43215/3830264213.py:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

